# Open Source GIS Pre-Processing Tutorial

## Table of Contents
1. [Create domain boundary shapefile](#Create-domain-boundary-shapefile)<br>
2. [Build geotiff from geogrid file](#Build-geotiff-from-geogrid-file)<br>
3. [Build routing stack](#Build-routing-stack)<br>
4. [Examine outputs of GIS pre-processor](#Examine-outputs-of-GIS-pre-processor)


## Create domain boundary shapefile

This tool takes an WRF Geogrid file and creates a single polygon shapefile that makes up the boundary of the domain of the M-grid (HGT_M, for example).

#### Get file paths

In [ ]:
import os
import Create_Domain_Boundary_Shapefile

cwd = os.getcwd()
data_folder = os.path.join(cwd, 'Croton_Lambert')
in_geogrid = os.path.join(data_folder, 'geo_em.d01.nc')
output_folder = os.path.join(cwd, 'Outputs')

#### Call help for the script on the command line

In [ ]:
! python Create_Domain_Boundary_Shapefile.py -h

#### Run the script on the command line

In [ ]:
! python Create_Domain_Boundary_Shapefile.py -i {in_geogrid} -o {output_folder}

#### View the output boundary shapefile

In [ ]:
import json
import geopandas
from ipyleaflet import Map, GeoJSON, ScaleControl, FullScreenControl, basemaps, SplitMapControl, basemap_to_tiles, LayersControl

boundary_shp = os.path.join(output_folder,'geo_em.d01_boundary.shp')
b_shp = geopandas.read_file(boundary_shp)
b_shp = b_shp.to_crs(epsg=4326)

b_json = os.path.join(output_folder, 'boundary.json')
b_shp.to_file(b_json, driver='GeoJSON')

with open(b_json, 'r') as f:
    data = json.load(f)
    
m = Map(center=(41.50, -73.73), zoom=10)

geo_json = GeoJSON(data=data, name='Domain boundary')
m.add_control(ScaleControl(position='bottomleft'))
m.add_control(FullScreenControl())

right_layer = basemap_to_tiles(basemap=basemaps.OpenStreetMap.Mapnik)
left_layer = basemap_to_tiles(basemap=basemaps.Esri.WorldImagery)

control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)
m.add_layer(geo_json)

control = LayersControl(position='topright')
m.add_control(control)

m

## Build geotiff from geogrid file

This is a program to export >=2D variables from a WRF-Hydro input file (geogrid or Fulldom_hires) file to an output raster format, with all spatial and coordinate system metadata. If a 3-dimensional variable is selected, individual raster bands will be created in the output raster for each index in the 3rd dimension. If a 4-dimensional variable is selected, the first index in the 4th dimension will be selected and the variable will be treated as a 3-dimensional variable described above.

#### Run the script using Python

In [ ]:
import rasterio
from matplotlib import pyplot
from Build_GeoTiff_From_Geogrid_File import build_geogrid_raster

in_geogrid = os.path.join(data_folder, 'geo_em.d01.nc')
output_folder = os.path.join(cwd, 'Outputs')
in_var = "HGT_M"
out_file = os.path.join(output_folder, f'{in_var}.tif')

build_geogrid_raster(in_geogrid, in_var, out_file)

#### View outputs

In [ ]:
import matplotlib
from osgeo import gdal
from ipyleaflet import ImageOverlay
import warnings
warnings.filterwarnings("ignore")
    

def show_raster_map(raster_path, map_id):
    # Make new folder if does not exist
    raster_outputs_prj = os.path.join(output_folder, 'Raster_Outputs_Prj')
    if not os.path.exists(raster_outputs_prj):
        os.mkdir(raster_outputs_prj)
    
    # Reproject raster
    input_raster = gdal.Open(raster_path)
    file_name = os.path.basename(raster_path)
    output_raster = os.path.join(raster_outputs_prj, file_name)
    gdal.Warp(output_raster, input_raster, dstSRS='EPSG:4326')
    
    # Open in rasterio, give colormap, make no data transparent, save as png
    src = rasterio.open(output_raster)
    var_name = os.path.basename(raster_path).replace(".tif","")

    if var_name in ['LATITUDE', 'frxst_pts']:
        cmap = pyplot.get_cmap('binary')
        pyplot.imshow(src.read(1), cmap=cmap,  aspect='auto', vmin=0.00001)
        cmap.set_under('k', alpha=0)
    elif var_name in ['CHANNELGRID']:
        cmap = pyplot.get_cmap('binary')
        pyplot.imshow(src.read(1), cmap=cmap,  aspect='auto', vmax=-1)
        cmap.set_under('k', alpha=0)
    elif var_name in ['LONGITUDE']:
        cmap = pyplot.get_cmap('binary')
        pyplot.imshow(src.read(1), cmap=cmap,  aspect='auto', vmax=-1)
        cmap.set_over('k', alpha=0)
    elif var_name in ['TOPOGRAPHY']:
        cmap = pyplot.get_cmap('BrBG')
        pyplot.imshow(src.read(1), cmap=cmap,  aspect='auto', vmin=1)
        cmap.set_under('k', alpha=0)
    elif var_name in ['FLOWDIRECTION']:
        colors = ['#5959a6', '#806c93', '#a65959', '#a68659', '#a6a659', '#93a659', '#669966', '#669988', '#999999']
        scale = [1, 2, 4, 8, 16, 32, 64, 128, 255]
        cmap = matplotlib.colors.ListedColormap(colors)
        norm = matplotlib.colors.BoundaryNorm(scale, len(colors))
        pyplot.imshow(src.read(1), cmap=cmap,  aspect='auto', norm=norm, interpolation='nearest', vmin=0.00001)
        cmap.set_under('k', alpha=0)
    elif var_name in ['landuse']:
        colors = ['#ed0000', '#dbd83d', '#aa7028', '#fbf65d', '#e2e2c1', '#ccba7c', '#dcca8f', '#fde9aa', '#68aa63', '#85c724',
                  '#38814e', '#1c6330', '#b5c98e', '#476ba0', '#70a3ba', '#bad8ea', '#b2ada3', '#c9c977', '#a58c30', '#d1ddf9',
                  '#d1ddf9']
        scale = [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23]
        cmap = matplotlib.colors.ListedColormap(colors)
        norm = matplotlib.colors.BoundaryNorm(scale, len(colors))
        pyplot.imshow(src.read(1), cmap=cmap,  aspect='auto', norm=norm, interpolation='nearest', vmin=0)
        cmap.set_under('k', alpha=0)
    elif var_name in ['STREAMORDER']:
        mask = data==241
        cmap = pyplot.get_cmap('Blues')
        pyplot.imshow(src.read(1), cmap=cmap,  aspect='auto', vmin=0.00001)
        cmap.set_under('k', alpha=0)
    else:
        cmap = pyplot.get_cmap('Blues')
        pyplot.imshow(src.read(1), cmap=cmap,  aspect='auto', vmin=1)
        cmap.set_under('k', alpha=0)
    
    pyplot.axis('off')
    new_file_name = file_name.replace('.tif', '.png')
    pyplot.savefig(os.path.join(raster_outputs_prj, new_file_name), transparent=True, bbox_inches='tight')
    
    # get bounds from boundary shapefile
    bounds_df = (b_shp.bounds)
    minx = bounds_df.iloc[0][0]
    miny = bounds_df.iloc[0][1]
    maxx = bounds_df.iloc[0][2]
    maxy = bounds_df.iloc[0][3]

    bounds = ((miny, minx), (maxy, maxx))
    
    # Overlay image on map
    map_name = new_file_name.replace(".png","")
    image = ImageOverlay(
        url=f'http://localhost:8888/tree/Outputs/Raster_Outputs_prj/{new_file_name}',
        bounds=bounds, name=map_name)
    
    map_id.add_layer(image)
    pyplot.close()

In [ ]:
m2 = Map(center=(41.50, -73.73), zoom=10)

m2.add_control(ScaleControl(position='bottomleft'))
m2.add_control(FullScreenControl())
control = LayersControl(position='topright')
m2.add_control(control)

m2

In [ ]:
show_raster_map(out_file, m2)

## Build routing stack

This is a program to perform the full routing-stack GIS pre-processing for WRF-Hydro. The inputs will be related to the domain, the desired routing nest factor, and other options and parameter values. The output will be a routing stack zip file with WRF-Hydro domain and parameter files.

• Required:<br>
&emsp;-WRF GEOGRID file (.nc)<br>
&emsp;-High-resolution Elevation<br>
&emsp;&emsp;-Elevation file (Esri GRID, GeoTIFF, etc.)<br>
&emsp;&emsp;-Mosaic Dataset<br>

• Parameters<br>
&emsp;-Regridding Factor – nesting relationship of routing:land grids<br>
&emsp;-Minimum basin size (in routing grid cells)<br>
&emsp;-OVROUGHRTFAC – constant<br>
&emsp;-RETDEPRTFAC – constant<br>
&emsp;-LKSATFAC – constant<br>

• Optional:<br>
&emsp;-Station Locations (.csv)<br>
&emsp;-Lake Polygons (polygon feature class or .shp)<br>

#### Using Python

In [ ]:
import shutil
from Build_Routing_Stack import GEOGRID_STANDALONE
from Build_Routing_Stack import varList2D
import wrfhydro_functions as wrfh

cwd = os.getcwd()
data_folder = os.path.join(cwd, 'Croton_Lambert')
output_folder = os.path.join(cwd, 'Outputs')

in_geogrid = os.path.join(data_folder, 'geo_em.d01.nc')
in_dem = os.path.join(data_folder, 'NED_30m_02b_Croton.tif')
out_zip = os.path.join(output_folder, 'croton_test.zip')
projdir = os.path.join(output_folder, 'scratch_dir')
regrid_factor = 10
routing_cells = 200

# Configure logging
logfile = out_zip.replace('.zip', '.log')
tee = wrfh.TeeNoFile(logfile, 'w')

# Create scratch directory for temporary outputs
projdir = os.path.join(output_folder, 'scratchdir')
projdir = os.path.abspath(projdir)
if os.path.exists(projdir):
    shutil.rmtree(projdir)
os.makedirs(projdir)

GEOGRID_STANDALONE(inGeogrid=in_geogrid, regridFactor=regrid_factor, inDEM=in_dem,
                   projdir=projdir, threshold=routing_cells, out_zip=out_zip, varList2D=varList2D )

tee.close()
del tee

#### Using command line

On the command line, type 'python' followed by the name of the script and the parameters need by the script. If you are unsure of what parameters are necessary, type '-h' to call the help message.

In [ ]:
os.chdir(cwd)
! python Build_Routing_Stack.py -h

### Examine outputs of GIS pre-processor

This tool takes the output zip file from the ProcessGeogrid script and creates a raster from each output NetCDF file. The Input should be a .zip file that was created using the WRF Hydro pre-processing tools. The tool will create the folder which will contain the results (out_folder), if that folder does not already exist.

In [ ]:
from Examine_Outputs_of_GIS_Preprocessor import examine_outputs
from wrfhydro_functions import ZipCompat

raster_outputs = os.path.join(output_folder, "Raster_Outputs")
if not os.path.exists(raster_outputs):
    os.mkdir(raster_outputs)

ZipCompat(out_zip).extractall(raster_outputs)
examine_outputs(raster_outputs, skipfiles=[])

In [ ]:
m3 = Map(center=(41.50, -73.73), zoom=10)

m3.add_control(ScaleControl(position='bottomleft'))
m3.add_control(FullScreenControl())
control = LayersControl(position='topright')
m3.add_control(control)

m3

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact

def f(x):
    r_path = os.path.join(raster_outputs, f"{x}.tif")
    show_raster_map(r_path, m3)

in_raster = widgets.Dropdown(
    options=[('Basin', 'BASIN'), ('Flow accumulation', 'FLOWACC'), ('Flow direction', 'FLOWDIRECTION'), 
             ('Land use', 'landuse'), ('Stream order', 'STREAMORDER'), ('Topography', 'TOPOGRAPHY')],
    value='FLOWDIRECTION',
    description='Variable:',
)
interact(f, x=in_raster)

In [ ]:
def see_raster(x):
    src = rasterio.open(os.path.join(raster_outputs, f"{x}.tif"))
    if x in ['LATITUDE', 'LONGITUDE']:
        cmap='binary'
    else:
        cmap='turbo'
    pyplot.imshow(src.read(1), cmap=cmap,  aspect='auto')
    pyplot.show()

in_raster = widgets.Dropdown(
    options=[('Basin', 'BASIN'), ('Basin mask', 'basn_msk'), ('Channel grid', 'CHANNELGRID'), ('Flow accumulation', 'FLOWACC'),
            ('Flow direction', 'FLOWDIRECTION'), ('Forecast points', 'frxst_pts'), ('Lake grid', 'LAKEGRID'),
            ('Land use', 'landuse'), ('Latitude', 'LATITUDE'), ('LKSATFAC', 'LKSATFAC'), ('Longitude', 'LONGITUDE'),
            ('OVROUGHRTFAC', 'OVROUGHRTFAC'), ('RETDEPRTFAC', 'RETDEPRTFAC'), ('Stream order', 'STREAMORDER'),
            ('Topography', 'TOPOGRAPHY')],
    value='FLOWACC',
    description='Variable:',
)

interact(see_raster, x=in_raster)